## 1 vs all training

This is almost the same as vc138 training, but use latest data and hand merge vc.

In [ ]:
import os
import sys
import json

import sys


import akagi
print("akagi version:", akagi.__version__)
from akagi.data_sources import RedshiftDataSource, S3DataSource
from akagi.iterator import FileFormat

import numpy as np

import pandas as pd
import glob

In [ ]:
import sys
sys.path.append("../")

In [ ]:
DATA_DIR = ""

BASE_DATA_DIR= os.path.normpath(os.path.join(DATA_DIR, ''))



In [ ]:
#Data dirs {train, validation}
TRAIN_DATA_DIR = os.path.normpath(os.path.join(BASE_DATA_DIR, "train"))
VALID_DATA_DIR = os.path.normpath(os.path.join(BASE_DATA_DIR, "valid"))

In [ ]:
BASE_MODEL_DIR = "./trained_model/inceptionv3/"
BASE_NAME = ""
CODE_EXTRACTOR=""

## Download code extractor from S3

In [ ]:
REGION_NAME = "ap-northeast-1"
DEFAULT_OUTPUT = "text"

os.environ['AWS_DEFAULT_REGION'] = REGION_NAME
os.environ['AWS_DEFAULT_OUTPUT'] = DEFAULT_OUTPUT

!aws s3 sync 

## hand merge category

In [ ]:
from category import VirtualCategories

In [ ]:
CATEGORY_DICT_CSV=os.path.join(BASE_DATA_DIR, "")

In [ ]:
from category import load_category_dict

In [ ]:
catdict = load_category_dict(CATEGORY_DICT_CSV)

In [ ]:
mergekeys = [

]

In [ ]:
[[catdict[int(key)] for key in mergekey.split("_")] for mergekey in mergekeys]

In [ ]:
orgkeyset = set(str(key) for key in catdict.keys())

In [ ]:
len(orgkeyset)

In [ ]:
nonmergedkey = orgkeyset

In [ ]:
for mergekey in mergekeys:
    mergekeyset = set(mergekey.split("_"))
    nonmergedkey = nonmergedkey - mergekeyset

In [ ]:
len(nonmergedkey)

In [ ]:
namedict = {key:catdict[int(key)] for key in nonmergedkey}

In [ ]:
for mergekey in mergekeys:
    first = mergekey.split("_")[0]
    namedict[mergekey] = catdict[int(first)]

In [ ]:
targetkeyset = set(nonmergedkey)

In [ ]:
for mergekey in mergekeys:
    targetkeyset.add(mergekey)

In [ ]:
len(targetkeyset)

In [ ]:
vcdict = {"keys":targetkeyset, "names":namedict}

In [ ]:
CATSTORE_PATH = ""

In [ ]:
import pickle

In [ ]:
with open(CATSTORE_PATH, "wb") as f:
    pickle.dump(vcdict, f)

In [ ]:
nonmergedkey

In [ ]:
vc = VirtualCategories(vcdict)

## Training setup

In [ ]:
from models.processor import create_generators

TRAIN_DATAGEN, VALID_DATAGEN = create_generators()

In [ ]:
from one_vs_all import create_multiple_model_trainer

def create_multimodeltrainer():
    return create_multiple_model_trainer(base_model_name = BASE_NAME, basedir = BASE_MODEL_DIR, traingen=TRAIN_DATAGEN, validgen=VALID_DATAGEN, extractor_path=CODE_EXTRACTOR)

In [ ]:
multitrainer = create_multimodeltrainer()

In [ ]:
from one_vs_all import setup_filedict_by_vc

In [ ]:
setup_filedict_by_vc(multitrainer, vc, TRAIN_DATA_DIR, VALID_DATA_DIR)
multitrainer.sample_valid_files(100, seed=123)

In [ ]:
catkeys = vc.keys()

In [ ]:
len(multitrainer.valid_files_dict[mergekeys[0]])

## Training

In [ ]:
multitrainer.train_categories(mergekeys[1:2], eachepochs=5)

In [ ]:
multitrainer.train_categories(mergekeys[2:5], eachepochs=5)

## Convert to code model and send result

In [ ]:
targetcats = mergekeys[8:]

In [ ]:
DIST_DIR=''

In [ ]:
from codeextractor import best_fullmodel_file_to_codemodel_file_by_basename
import tqdm

In [ ]:
[best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, key) for key in tqdm.tqdm(targetcats)]

In [ ]:
os.makedirs(DIST_DIR, exist_ok=True)

In [ ]:
from codeextractor import best_code_full_path

In [ ]:
def link_codemodel_to(targetkey):
    codejson, codeweight, _, _ = best_code_full_path(BASE_MODEL_DIR, BASE_NAME, targetkey)
    os.link(codejson, os.path.join(DIST_DIR, os.path.basename(codejson)))
    os.link(codeweight, os.path.join(DIST_DIR, os.path.basename(codeweight)))
    return targetkey

In [ ]:
[link_codemodel_to(key) for key in targetcats]

In [ ]:
S3PATH = "" + DIST_DIR

In [ ]:
# !aws s3 cp --dryrun --exclude "*" --include "code*" {DIST_DIR} {S3PATH} --recursive 
!aws s3 cp --exclude "*" --include "code*" {DIST_DIR} {S3PATH} --recursive

### Create predict df

In [ ]:
BASE_CODE_NAME = "code_" + BASE_NAME

In [ ]:
fdict = vc.list_dict(VALID_DATA_DIR)

In [ ]:
from one_vs_all import ModelBinder
from codeextractor import DNNCodeExtractor

def ModelBinder_create(base_model_name = BASE_CODE_NAME, basedir = BASE_MODEL_DIR, extractor_path=CODE_EXTRACTOR):
    return ModelBinder(base_model_name, basedir, DNNCodeExtractor.create_from(extractor_path))

In [ ]:
ftuppls = [(key, file) for key in fdict.keys() for file in fdict[key]]
labels = [tup[0] for tup in ftuppls]
paths = [tup[1] for tup in ftuppls]

In [ ]:
len(labels), len(paths)

In [ ]:
ens = ModelBinder_create()

In [ ]:
ens.load_all_models(catkeys)

In [ ]:
%%time
df = ens.predict(paths)

In [ ]:
df['label'] = labels

In [ ]:
VALID_DF_PATH=""

In [ ]:
df.to_pickle(VALID_DF_PATH)

### Check result

In [ ]:
from visualize import plot_image_list

In [ ]:
def plot_cat(df, cat, thrld, origin=0):
    filtered = df[df[cat] > thrld].sort_values(by=cat, ascending=False)
    for i in range(0, 50, 5):
        print(",".join([os.path.basename(os.path.dirname(v)) for v in filtered['filepaths'].values[i+origin:(i+5+origin)]]))
    for i in range(0, 50, 5):
        print(",".join(["{0:.4f}".format(v) for v in filtered[cat].values[origin+i:(origin+5+i)]]))
    plot_image_list(filtered["filepaths"].values[origin:(origin+50)])

In [ ]:
mergekeys_str

In [ ]:
# plot_cat(df, mergekeys_str, 0.8)

In [ ]:
# plot_cat(df, mergekeys_str, 0.8, 100)

### Hand trial

In [ ]:
mergekeys_str = '11_13_54_120_130_58_86_96'

In [ ]:
mergekeyset = set(key for key in mergekeys_str.split('_'))

In [ ]:
mergekeyset

In [ ]:
targetkeyset = orgkeyset

In [ ]:
targetkeyset = orgkeyset-mergekeyset

In [ ]:
targetkeyset.add(mergekeys_str)

In [ ]:
len(targetkeyset)

In [ ]:
namedict = {key:catdict[int(key)] for key in (orgkeyset - mergekeyset)}

In [ ]:
namedict[mergekeys_str] = "カレー"

In [ ]:
vcdict = {"keys":targetkeyset, "names":namedict}

In [ ]:
CATSTORE_PATH = ""

In [ ]:
import pickle

In [ ]:
with open(CATSTORE_PATH, "wb") as f:
    pickle.dump(vcdict, f)

In [ ]:
vc = VirtualCategories(vcdict)

In [ ]:
vc.name(mergekeys_str)

In [ ]:
multitrainer.train_one(mergekeys_str, eachepochs=10)

In [ ]:
mergekeys[0]

In [ ]:
from codeextractor import best_fullmodel_file_to_codemodel_file_by_basename
best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, mergekeys_str)